In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from tqdm.auto import tqdm
from ydata_profiling import ProfileReport

In [ ]:
import mlflow

In [ ]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db
# mlflow server --host 127.0.0.1 --port 5000
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("drinking")

In [ ]:
tqdm.pandas()
warnings.filterwarnings('ignore')

In [ ]:
path = 'Artefacts/'

In [ ]:
df = pd.read_parquet('Data/smoking_drinking.parquet')

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [ ]:
profile

In [ ]:
profile.to_file(f"{path}profile.html")

In [ ]:
def plot_boxplots(dataframe):
    num_columns = dataframe.select_dtypes(include=['number']).columns 
    num_plots = len(num_columns)
    rows = (num_plots + 1) // 2 

    fig, axes = plt.subplots(nrows=rows, ncols=2, figsize=(20, 20))  

    for i, column in enumerate(num_columns):
        row = i // 2
        col = i % 2
        ax = axes[row, col]
        sns.boxplot(x=dataframe[column], ax=ax)
        ax.set_title(f"Boxplot of {column}")
        ax.set_xlabel(column)

    plt.tight_layout()
    plt.show()
    return fig

In [ ]:
box_plot = plot_boxplots(df)

In [ ]:
box_plot

In [ ]:
box_plot.savefig(f"{path}box_plot.jpeg") 

In [ ]:
def remove_outliers(df, columns, k=1.5):
    for column in columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75) 
        iqr = q3 - q1
        df[column] = df[column].clip(lower=q1 - k * iqr, upper=q3 + k * iqr)

    return df

In [ ]:
remove_outliers(df,['waistline','SBP', 'DBP','BLDS','tot_chole','triglyceride','serum_creatinine','SGOT_AST', 'SGOT_ALT'],k=1.5)

In [ ]:
fig = go.Figure()

for category in df['DRK_YN'].unique():
    data = df[df['DRK_YN'] == category]
    fig.add_trace(go.Histogram(x=data['age'], name=category, nbinsx=int(df.age.max())))  
    
fig.update_layout(
    title='Histogram of Age by Drinking',
    xaxis_title='Age',
    yaxis_title='Count',
)

fig.show()

In [ ]:
fig.write_html(f"{path}/hist_age_drkn.html")

In [ ]:
fig1 = px.histogram(df, x="sex", color="DRK_YN", barmode="group")

fig.update_layout(
    xaxis_title="sex",
    yaxis_title="Count",
    barmode="group",
    bargap=0.1
)

for data in fig.data:
    fig.add_annotation(
        x=data.x,
        y=data.y,
        text=data.y,
#         showarrow=False,
        font=dict(size=12)
)

fig1.show()

In [ ]:
fig1.write_html(f"{path}/hist_sex_drkn.html")

In [ ]:
i = random.randint(0, 93)
colorscales = px.colors.named_colorscales()
corr = px.imshow(df.corr(numeric_only=True), 
                            text_auto=True, width=800, height=800, 
                            color_continuous_scale=colorscales[i],)
corr.show()

In [ ]:
corr.write_html(f"{path}/corr.html")

In [ ]:
df = df[(df.sight_left <= 2) & (df.sight_right <= 2)]

In [ ]:
df = df.drop_duplicates() # Removing duplicates

In [ ]:
df["hearing"] = df.apply(lambda row: "Normal" if row["hear_right"] == 1 and row["hear_left"] == 1 else "Abnormal", axis=1)
df = df.drop(["hear_left", "hear_right"], axis=1)
df.head()

In [ ]:
df = df.drop(df[df.SMK_stat_type_cd == 2].index) 
smoking_replace = {
    1: "N",
    3: "Y"
}
df.SMK_stat_type_cd = df.SMK_stat_type_cd.replace(smoking_replace)

In [ ]:
df_control = df.query("SMK_stat_type_cd == 'N' and DRK_YN == 'N'")
df_control = df_control.drop(["SMK_stat_type_cd", "DRK_YN"], axis=1)
df_control.describe()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(16, 8))

sns.lineplot(data=df_control, x="age", y="height", hue="sex", ax=axs[0][0])
sns.lineplot(data=df_control, x="age", y="weight", hue="sex", ax=axs[0][1])
sns.lineplot(data=df_control, x="age", y="sight_right", hue="sex", ax=axs[0][2])
sns.lineplot(data=df_control, x="age", y="sight_left", hue="sex", palette={"Male":"black", "Female":"green"}, ax=axs[0][2])
axs[0][2].set_ylabel("sight")

sns.lineplot(data=df_control, x="weight", y="height", hue="sex", ax=axs[1][0])
sns.lineplot(data=df_control, x="weight", y="waistline", hue="sex", ax=axs[1][1])
sns.lineplot(data=df_control, x="weight", y="sight_right", hue="sex", ax=axs[1][2])
sns.lineplot(data=df_control, x="weight", y="sight_left", hue="sex", palette={"Male":"black", "Female":"green"}, ax=axs[1][2])
axs[1][2].set_ylabel("sight")

plt.show()

In [ ]:
fig.savefig(f"{path}line_plot.jpeg") 

## Análise dos atributos físicos de não fumantes e não bebedores

1. Em média, os homens são mais altos que as mulheres; as mulheres também parecem ficar mais baixas à medida que envelhecem, enquanto que nos homensuasua altusra atinmge o pico por volta dos 30 anos e depois lentamente começam a ficar mais baixas com a idade também:. existe uma correlação negativa entre idade e altura

2. O peso das mulheres parece ser consistente, pois oscila entre 55kg e 58kg, caindo drasticamente após os 70 anos. Por outro lado, o peso dos homens aumenta até aos 35 anos e depois o seu peso diminui a um ritmo mais rápido do que o peso das mulheres ; isso pode ser causado pelo fato de que o homem médio começa a produzir menos testosterona após os 35 anos

3. Este conjunto de dados mostra que o homem saudável médio tem uma visão melhor do que a mulher média. No entanto, a visão de ambos os sexos parece permanecer consistente até os 35 anos de idade, e depois parece que piora progressivamente.

4. O conjunto de dados mostra que o homem médio é mais alto do que a mulher média; peso e altura parecem estar positivamente correlacionados.

5. A cintura e o peso também parecem estar positivamente correlacionados. No entanto, a relação entre peso e cintura parece ser muito semelhante tanto para homens quanto para mulheres.

6. Não parece haver qualquer correlação entre peso e visão; a ligeira queda no início deve-se ao fato de os idosos terem baixa massa corporal e pior visão, como mostrado em um gráfico anterior. A razão pela qual quis comparar o peso e a visão é porque levantei a hipótese de que pessoas mais pesadas teriam maior probabilidade de ter diabetes, o que afetaria a visão.

# Efeitos a longo prazo da bebida no corpo

Agora, vou pegar uma amostra dpessoas que bebeme mass, que nunca fumaram, e ver como só a bebida afeta sseu corspoD depois vou comparar os dados que encontrei com os do grupo de controle.

In [ ]:
df_drink = df[(df.SMK_stat_type_cd == "N") & (df.DRK_YN == "Y")]
df_drink.head()

### Barriga de cerveja

Normalmente, quando um alcoólatra envelhece, fica com barriga de cerveja devido ao excesso de calorias do álcool. Nesta seção da minha análise quero provar isso.

As pessoas começam a ficar com barriga de cerveja depois dos 35 anos, então vou analisar esse substrato dos alcoólatras.

In [ ]:
bb_df = df_drink[df_drink.age >= 35]
bb_df.head()

In [ ]:
control_for_bb = df_control[df_control.age >= 35]

In [ ]:
bb_bmi = bb_df.weight/((bb_df.height/100) ** 2)
normal_bmi = control_for_bb.weight/((control_for_bb.height/100) ** 2)

sns.lineplot(x=control_for_bb.age, y=normal_bmi, label="Control")
sns.lineplot(x=bb_df.age, y=bb_bmi, label="Alcoholics")
plt.xlabel("Age")
plt.ylabel("BMI")
plt.title("Comparing ratio of weight per waistline against age")
plt.legend()
plt.savefig(f"{path}bmi_comparison_plot.jpeg")
plt.show()

### Conclusão - barriga de cerveja

Há evidências que sugerem que as pessoas desenvolvem barriga de cerveja, pois o gráfico acima mostra que os alcoólatras tendem a ter um IMC mais elevado do que o grupo de controle, provavelmente devido ao excesso de calorias do álcool. No entanto, por volta dos 65 anos, tanto o IMC do grupo de controlo como do grupo alcoólatra parece ser muito semelhante e parece estar a diminuir rapidamente; isso pode ser devido a várias condições subjacentes que uma pessoa idosa pode ter.

# Efeitos a longo prazo do fumo e da bebida no corpo

Tanto fumar quanto beber afetam a audição, a visão e a vida, por isso analisarei juntos esses efeitos no corpo.

In [ ]:
df_sd = df[(df.SMK_stat_type_cd == "Y") & (df.DRK_YN == "Y")]
df_sd.head()

## Perda de audição
E existem algumas correlações entre fumar e beber e perda auditiva. Quero provar isso com este conjunto de dados.

In [ ]:
sns.countplot(x=df_control.hearing)
sns.countplot(x=df_sd.hearing, palette={"green", "purple"})
plt.title("Hearing for control group and smoking and drinking group")
plt.savefig(f"{path}Hearing.jpeg")
plt.show()

In [ ]:
control_normal = df_control.hearing.value_counts()["Normal"]
control_abnormal = df_control.hearing.value_counts()["Abnormal"]
control_ratio = control_normal/(control_abnormal + control_normal)

sd_normal = df_sd.hearing.value_counts()["Normal"]
sd_abnormal = df_sd.hearing.value_counts()["Abnormal"]
sd_ratio = sd_normal/(sd_abnormal + sd_normal)

print("(Higher percentage = better)")
print(f"The percentage of people in the control group with normal hearing is {int(control_ratio * 100)}%")
print(f"The percentage of people in the drinking and smoking group with normal hearing is {int(sd_ratio * 100)}%")

 ### Conclusão sobre perda auditiva Os dados não apoiam a afirmação acima de que beber e fumar são mais prejudiciais para a audição, uma vez que existe uma percentagem maior de fumadores e bebedores que têm melhor audição do que o grupo de controlo.

 A razão pela qual a percentagem de pessoas no grupo de controle com audição normal é menor pode ser devido a outras condições subjacentes que possam ter.

## Força de visão.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

sns.lineplot(data=df_control, x="age", y="sight_left", label="Control", ax=axs[0])
sns.lineplot(data=df_sd, x="age", y="sight_left", label="Smoking and drinking", ax=axs[0])
axs[0].set_title("Left eye")

sns.lineplot(data=df_control, x="age", y="sight_right", label="Control", ax=axs[1])
sns.lineplot(data=df_sd, x="age", y="sight_right", label="Smoking and drinking", ax=axs[1])
axs[1].set_title("Right eye")
plt.savefig(f"{path}eye_sight.jpeg")
plt.show()

In [ ]:
mlflow.log_artifact(f'{path}')